# Quantile regression of forecast accuracy

gde 7/10/2018
Initial model estimation of range of actual forecasts as a function of predicted

In [1]:
import pandas as pd #for working with dataframe
import numpy as np
from scipy import stats, integrate #will help in stats operation in histograms
import matplotlib.pyplot as plt #to plot the PE
import seaborn as sns #nice looking plots
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg


%matplotlib inline
sns.set(color_codes=True)

pd.options.display.max_columns=50

In [23]:
flatdata_adjusted=pd.read_csv('flatdata.csv')

In [9]:
def UnifiedImprovement(x): #to adjust the forecast
    if x.ImprovementType<7:
        unifiedtype=1
    elif x.ImprovementType<12:
        unifiedtype=2
    else:
        unifiedtype=3
    return unifiedtype
flatdata_adjusted=flatdata_adjusted.assign(UnifiedImprovementType=flatdata_adjusted.apply(UnifiedImprovement, axis=1))

In [10]:
pd.options.display.max_columns=50
flatdata_adjusted

,BriefDescription,ProjectYear,InternalProjectID,ImprovementType,AreaTypeFuncClass,main_ProjectInformation.State,TollType,TrafficForecast,ForecastYear,ForecastYearType,YearForecastProduced,ForecastingAgency,ForecastMethodology,SegmentDescription,ProjectID,SegmentID,CountID,Year_ofObservation,Count,Station_Identifier,State,CounterID,FunctionalClass,AdjustedForecast,Error,AbsError,PercentError,AbsPE,FunctionalClassName,MethodologyName,AreaTypeName,UnemploymentRateOpeningYear,UnemploymentRate_YearProduced,UnifiedImprovementType
0,NaN,2006.0,2320,12,3.0,WI,No Tolls on 1+ lane,34000,2007,1,2006,1.0,1,NaN,58-6302-2320-,NaN,58-22417-401399,2007,31500,401399.0,58,58-22417-401399,3,34000.0,-2500.0,2500.0,-7.352941,7.352941,Principle Arterial,Traffic Count Trend,Urban,4.9,4.7,3
1,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,30000,2011,1,2007,1.0,1,NaN,58-6303-2467-,NaN,58-22419-404512,2011,22200,404512.0,58,58-22419-404512,3,30000.0,-7800.0,7800.0,-26.000000,26.000000,Principle Arterial,Traffic Count Trend,Urban,7.8,4.9,3
2,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,19000,2011,1,2007,1.0,1,NaN,58-6304-2467-,NaN,58-22421-404513,2011,20100,404513.0,58,58-22421-404513,3,19000.0,1100.0,1100.0,5.789474,5.789474,Principle Arterial,Traffic Count Trend,Urban,7.8,4.9,3
3,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,21900,2011,1,2007,1.0,1,NaN,58-6305-2467-,NaN,58-22423-404524,2011,18400,404524.0,58,58-22423-404524,4,21900.0,-3500.0,3500.0,-15.981735,15.981735,Minor Arterial,Traffic Count Trend,Urban,7.8,4.9,3
4,NaN,2006.0,2322,12,3.0,WI,No Tolls on 1+ lane,22700,2009,1,2006,1.0,1,NaN,58-6306-2322-,NaN,58-22425-401843,2009,22600,401843.0,58,58-22425-401843,3,22700.0,-100.0,100.0,-0.440529,0.440529,Principle Arterial,Traffic Count Trend,Urban,8.6,4.7,3
5,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,2800,2011,1,2007,1.0,1,NaN,58-6307-2467-,NaN,58-22427-404541,2011,2500,404541.0,58,58-22427-404541,5,2800.0,-300.0,300.0,-10.714286,10.714286,Major Collector,Traffic Count Trend,Urban,7.8,4.9,3
6,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,33500,2011,1,2007,1.0,1,NaN,58-6308-2467-,NaN,58-22429-404545,2011,33500,404545.0,58,58-22429-404545,3,33500.0,0.0,0.0,0.000000,0.000000,Principle Arterial,Traffic Count Trend,Urban,7.8,4.9,3
7,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,1500,2011,1,2007,1.0,1,NaN,58-6309-2467-,NaN,58-22431-404547,2011,2200,404547.0,58,58-22431-404547,7,1500.0,700.0,700.0,46.666667,46.666667,Local,Traffic Count Trend,Urban,7.8,4.9,3
8,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,23800,2011,1,2007,1.0,1,NaN,58-6310-2467-,NaN,58-22433-404548,2011,25400,404548.0,58,58-22433-404548,3,23800.0,1600.0,1600.0,6.722689,6.722689,Principle Arterial,Traffic Count Trend,Urban,7.8,4.9,3
9,NaN,2006.0,2373,12,1.0,WI,No Tolls on 1+ lane,8600,2008,1,2006,1.0,1,NaN,58-6311-2373-,NaN,58-22435-510108,2008,6600,510108.0,58,58-22435-510108,3,8600.0,-2000.0,2000.0,-23.255814,23.255814,Principle Arterial,Traffic Count Trend,Rural,4.9,4.7,3


In [11]:
# take some logs
flatdata_adjusted['Count_log'] = np.log(flatdata_adjusted['Count'])
flatdata_adjusted['AdjustedForecast_log'] = np.log(flatdata_adjusted['AdjustedForecast'])

flatdata_adjusted['Count_over_AdjustedForecast'] = flatdata_adjusted['Count'] / flatdata_adjusted['AdjustedForecast']


flatdata_adjusted['Error'] = flatdata_adjusted['Count'] - flatdata_adjusted['AdjustedForecast']
flatdata_adjusted['PercentError'] = (flatdata_adjusted['Count'] - flatdata_adjusted['AdjustedForecast']) / flatdata_adjusted['AdjustedForecast']

In [6]:
flatdata_adjusted.head()

,BriefDescription,ProjectYear,InternalProjectID,ImprovementType,AreaTypeFuncClass,main_ProjectInformation.State,TollType,TrafficForecast,ForecastYear,ForecastYearType,YearForecastProduced,ForecastingAgency,ForecastMethodology,SegmentDescription,ProjectID,SegmentID,CountID,Year_ofObservation,Count,Station_Identifier,State,CounterID,FunctionalClass,AdjustedForecast,Error,AbsError,PercentError,AbsPE,FunctionalClassName,MethodologyName,AreaTypeName,UnemploymentRateOpeningYear,UnemploymentRate_YearProduced,UnifiedImprovementType,Count_log,AdjustedForecast_log,Count_over_AdjustedForecast
0,NaN,2006.0,2320,12,3.0,WI,No Tolls on 1+ lane,34000,2007,1,2006,1.0,1,NaN,58-6302-2320-,NaN,58-22417-401399,2007,31500,401399.0,58,58-22417-401399,3,34000.0,-2500.0,2500.0,-0.073529,7.352941,Principle Arterial,Traffic Count Trend,Urban,4.9,4.7,3,10.357743,10.434116,0.926471
1,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,30000,2011,1,2007,1.0,1,NaN,58-6303-2467-,NaN,58-22419-404512,2011,22200,404512.0,58,58-22419-404512,3,30000.0,-7800.0,7800.0,-0.260000,26.000000,Principle Arterial,Traffic Count Trend,Urban,7.8,4.9,3,10.007848,10.308953,0.740000
2,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,19000,2011,1,2007,1.0,1,NaN,58-6304-2467-,NaN,58-22421-404513,2011,20100,404513.0,58,58-22421-404513,3,19000.0,1100.0,1100.0,0.057895,5.789474,Principle Arterial,Traffic Count Trend,Urban,7.8,4.9,3,9.908475,9.852194,1.057895
3,NaN,2007.0,2467,12,3.0,WI,No Tolls on 1+ lane,21900,2011,1,2007,1.0,1,NaN,58-6305-2467-,NaN,58-22423-404524,2011,18400,404524.0,58,58-22423-404524,4,21900.0,-3500.0,3500.0,-0.159817,15.981735,Minor Arterial,Traffic Count Trend,Urban,7.8,4.9,3,9.820106,9.994242,0.840183
4,NaN,2006.0,2322,12,3.0,WI,No Tolls on 1+ lane,22700,2009,1,2006,1.0,1,NaN,58-6306-2322-,NaN,58-22425-401843,2009,22600,401843.0,58,58-22425-401843,3,22700.0,-100.0,100.0,-0.004405,0.440529,Principle Arterial,Traffic Count Trend,Urban,8.6,4.7,3,10.025705,10.030120,0.995595


In [6]:
list(flatdata_adjusted)

['BriefDescription',
 'ProjectYear',
 'InternalProjectID',
 'ImprovementType',
 'AreaTypeFuncClass',
 'main_ProjectInformation.State',
 'TollType',
 'TrafficForecast',
 'ForecastYear',
 'ForecastYearType',
 'YearForecastProduced',
 'ForecastingAgency',
 'ForecastMethodology',
 'SegmentDescription',
 'ProjectID',
 'SegmentID',
 'CountID',
 'Year_ofObservation',
 'Count',
 'Station_Identifier',
 'main_TrafficCounts.State',
 'CounterID',
 'FunctionalClass',
 'AdjustedForecast',
 'Error',
 'AbsError',
 'PercentError',
 'AbsPE',
 'DiffYear',
 'UnifiedImprovementType',
 'Count_log',
 'AdjustedForecast_log',
 'Count_over_AdjustedForecast']

## Ordinary Least Squares (based on mean)

In [12]:
# start by estimating a basic regression model
mod = smf.ols(formula='Count \
                       ~ AdjustedForecast', data=flatdata_adjusted)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  Count   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                 3.044e+04
Date:                Wed, 11 Jul 2018   Prob (F-statistic):               0.00
Time:                        09:38:27   Log-Likelihood:                -40017.
No. Observations:                3807   AIC:                         8.004e+04
Df Residuals:                    3805   BIC:                         8.005e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         1454.5363    199.392  

## Least Absolute Deviation (based on median)

In [13]:
mod = smf.quantreg('Count \
                    ~ AdjustedForecast', data=flatdata_adjusted)
res = mod.fit(q=.5)
print(res.summary())


                         QuantReg Regression Results                          
Dep. Variable:                  Count   Pseudo R-squared:               0.7252
Model:                       QuantReg   Bandwidth:                       1079.
Method:                 Least Squares   Sparsity:                        5850.
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:38:50   Df Residuals:                     3805
                                        Df Model:                            1
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -36.3502     65.597     -0.554      0.580    -164.959      92.258
AdjustedForecast     0.9472      0.002    544.976      0.000       0.944       0.951

The condition number is large, 5.22e+04. This might indicate that there are
strong multicollinearity or ot

## Quantile regression (for different percentiles)

In [14]:
mod = smf.quantreg('Count \
                    ~ AdjustedForecast', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:                  Count   Pseudo R-squared:               0.4333
Model:                       QuantReg   Bandwidth:                       2223.
Method:                 Least Squares   Sparsity:                    2.236e+04
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:39:18   Df Residuals:                     3805
                                        Df Model:                            1
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         -902.7881    110.832     -8.146      0.000   -1120.085    -685.492
AdjustedForecast     0.6329      0.003    213.217      0.000       0.627       0.639

The condition number is large, 5.22e+04. This might indicate that there are
strong multicollinearity or ot

In [15]:
# now try a log-log model

mod = smf.quantreg('Count_log \
                    ~ AdjustedForecast_log', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:              Count_log   Pseudo R-squared:               0.6863
Model:                       QuantReg   Bandwidth:                     0.06272
Method:                 Least Squares   Sparsity:                        5.077
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:39:49   Df Residuals:                     3805
                                        Df Model:                            1
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.9876      0.131     -7.568      0.000      -1.243      -0.732
AdjustedForecast_log     1.0412      0.013     77.764      0.000       1.015       1.067
                         QuantReg Regression Results                          
Dep. Variabl

In [11]:
# now try a log-log model

mod = smf.quantreg('Count_log \
                    ~ AdjustedForecast_log \
                   -1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:              Count_log   Pseudo R-squared:               0.6788
Model:                       QuantReg   Bandwidth:                     0.06577
Method:                 Least Squares   Sparsity:                        5.525
Date:                Tue, 10 Jul 2018   No. Observations:                 3807
Time:                        16:54:54   Df Residuals:                     3806
                                        Df Model:                            1
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
AdjustedForecast_log     0.9372      0.002    475.128      0.000       0.933       0.941
                         QuantReg Regression Results                          
Dep. Variable:              Count_log   Pseudo R-squared:               0.7542
Model:                

In [16]:
# make the dependent variable the count / forecast

mod = smf.quantreg('Count_over_AdjustedForecast ~ 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)


                              QuantReg Regression Results                              
Dep. Variable:     Count_over_AdjustedForecast   Pseudo R-squared:            1.629e-07
Model:                                QuantReg   Bandwidth:                     0.05608
Method:                          Least Squares   Sparsity:                        2.842
Date:                         Wed, 11 Jul 2018   No. Observations:                 3807
Time:                                 09:40:17   Df Residuals:                     3806
                                                 Df Model:                            0
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5523      0.010     54.932      0.000       0.533       0.572
                              QuantReg Regression Results                              
Dep. Variable:     Count_over_AdjustedForecast   Pseudo R-s

In [17]:
# make percent error the dependent variable -- this gives the same results as the ratio

mod = smf.quantreg('PercentError ~ 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)


                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:            1.629e-07
Model:                       QuantReg   Bandwidth:                     0.05608
Method:                 Least Squares   Sparsity:                        2.842
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:40:33   Df Residuals:                     3806
                                        Df Model:                            0
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4477      0.010    -44.529      0.000      -0.467      -0.428
                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:           -1.403e-07
Model:                       QuantReg   Bandwidth:  

## Start to add some variables

### 2. forecast volume

In [18]:
# make percent error the dependent variable -- this gives the same results as the ratio

mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast \
                   + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)


                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:             0.009201
Model:                       QuantReg   Bandwidth:                     0.06071
Method:                 Least Squares   Sparsity:                        2.617
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:40:51   Df Residuals:                     3805
                                        Df Model:                            1
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.4901      0.014    -34.426      0.000      -0.518      -0.462
AdjustedForecast  1.866e-06    4.2e-07      4.448      0.000    1.04e-06    2.69e-06

The condition number is large, 5.22e+04. This might indicate that there are
strong multicollinearity or ot

In [34]:
# try log of adjusted forecast

mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast_log \
                   + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)


                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:             0.003104
Model:                       QuantReg   Bandwidth:                     0.05734
Method:                 Least Squares   Sparsity:                        2.801
Date:                Tue, 10 Jul 2018   No. Observations:                 3807
Time:                        17:32:38   Df Residuals:                     3805
                                        Df Model:                            1
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.6599      0.072     -9.166      0.000      -0.801      -0.519
AdjustedForecast_log     0.0223      0.007      3.018      0.003       0.008       0.037
                         QuantReg Regression Results                          
Dep. Variabl

### 3. Functional Class

In [19]:
# Add the functional class.  Reference class is interstate.  (There are no ramps observed)

flatdata_adjusted['FC_Interstate']        = flatdata_adjusted['FunctionalClass'].apply(lambda x : 1 if x==1 else 0)
flatdata_adjusted['FC_Ramp']              = flatdata_adjusted['FunctionalClass'].apply(lambda x : 1 if x==2 else 0)
flatdata_adjusted['FC_PrincipalArterial'] = flatdata_adjusted['FunctionalClass'].apply(lambda x : 1 if x==3 else 0)
flatdata_adjusted['FC_MinorArterial']     = flatdata_adjusted['FunctionalClass'].apply(lambda x : 1 if x==4 else 0)
flatdata_adjusted['FC_CollectorLocal']    = flatdata_adjusted['FunctionalClass'].apply(lambda x : 1 if (x>=5 and x<=7) else 0)
flatdata_adjusted['FC_Unknown']           = flatdata_adjusted['FunctionalClass'].apply(lambda x : 1 if x==8 else 0)

mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast \
                    + FC_PrincipalArterial \
                    + FC_MinorArterial \
                    + FC_CollectorLocal \
                    + FC_Unknown \
                    + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)


                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:              0.04761
Model:                       QuantReg   Bandwidth:                     0.06937
Method:                 Least Squares   Sparsity:                        1.983
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:42:53   Df Residuals:                     3801
                                        Df Model:                            5
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.2400      0.035     -6.788      0.000      -0.309      -0.171
AdjustedForecast     -4.418e-07   4.42e-07     -1.001      0.317   -1.31e-06    4.24e-07
FC_PrincipalArterial    -0.1333      0.033     -4.067      0.000      -0.198      -0.069
FC

### 4. Area Type

In [20]:
# area type -- urban is reference class

flatdata_adjusted['AT_Rural']   = flatdata_adjusted['AreaTypeFuncClass'].apply(lambda x : 1 if x>=2 else 0)
flatdata_adjusted['AT_Urban']   = flatdata_adjusted['AreaTypeFuncClass'].apply(lambda x : 1 if x==3 else 0)
flatdata_adjusted['AT_Unknown'] = flatdata_adjusted['AreaTypeFuncClass'].apply(lambda x : 1 if x==4 else 0)

mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast \
                    + FC_PrincipalArterial \
                    + FC_MinorArterial \
                    + FC_CollectorLocal \
                    + FC_Unknown \
                    + AT_Rural \
                    + AT_Unknown \
                    + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:              0.05501
Model:                       QuantReg   Bandwidth:                     0.06968
Method:                 Least Squares   Sparsity:                        1.830
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:43:21   Df Residuals:                     3799
                                        Df Model:                            7
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.2649      0.031     -8.617      0.000      -0.325      -0.205
AdjustedForecast     -8.967e-08    3.9e-07     -0.230      0.818   -8.54e-07    6.75e-07
FC_PrincipalArterial    -0.1254      0.030     -4.164      0.000      -0.184      -0.066
FC

### 5. Type of Project

In [21]:
# type of project -- existing road is reference class

flatdata_adjusted['IT_ExistingRoad'] = flatdata_adjusted['UnifiedImprovementType'].apply(lambda x : 1 if x==1 else 0)
flatdata_adjusted['IT_NewRoad']      = flatdata_adjusted['UnifiedImprovementType'].apply(lambda x : 1 if x==2 else 0)
flatdata_adjusted['IT_Unknown']      = flatdata_adjusted['UnifiedImprovementType'].apply(lambda x : 1 if x==3 else 0)

mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast \
                    + FC_PrincipalArterial \
                    + FC_MinorArterial \
                    + FC_CollectorLocal \
                    + FC_Unknown \
                    + AT_Rural \
                    + AT_Unknown \
                    + IT_NewRoad \
                    + IT_Unknown \
                    + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:              0.05582
Model:                       QuantReg   Bandwidth:                     0.06955
Method:                 Least Squares   Sparsity:                        1.826
Date:                Wed, 11 Jul 2018   No. Observations:                 3807
Time:                        09:43:59   Df Residuals:                     3797
                                        Df Model:                            9
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.2707      0.032     -8.432      0.000      -0.334      -0.208
AdjustedForecast     -2.595e-08   3.94e-07     -0.066      0.948   -7.99e-07    7.47e-07
FC_PrincipalArterial    -0.1220      0.031     -3.994      0.000      -0.182      -0.062
FC

### 7. Opening Year

### 8. year forecast produced

### 9. Forecast Horizon

In [19]:

mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast \
                    + FC_PrincipalArterial \
                    + FC_MinorArterial \
                    + FC_CollectorLocal \
                    + FC_Unknown \
                    + AT_Rural \
                    + AT_Unknown \
                    + IT_NewRoad \
                    + IT_Unknown \
                    + DiffYear \
                    + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:              0.05650
Model:                       QuantReg   Bandwidth:                     0.06950
Method:                 Least Squares   Sparsity:                        1.828
Date:                Tue, 10 Jul 2018   No. Observations:                 3807
Time:                        17:11:02   Df Residuals:                     3796
                                        Df Model:                           10
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.2710      0.032     -8.430      0.000      -0.334      -0.208
AdjustedForecast     -2.343e-08   3.95e-07     -0.059      0.953   -7.98e-07    7.51e-07
FC_PrincipalArterial    -0.1253      0.031     -4.105      0.000      -0.185      -0.065
FC

### 10. Unemployment Rate in Opening Year

### 11. Unemployment Rate in Forecast Year

### 12. Forecast Type

In [23]:
# Forecast methodology -- traffic count trend is reference

flatdata_adjusted['FM_CountTrend']     = flatdata_adjusted['ForecastMethodology'].apply(lambda x : 1 if x==1 else 0)
flatdata_adjusted['FM_PopGrowthRates'] = flatdata_adjusted['ForecastMethodology'].apply(lambda x : 1 if x==2 else 0)
flatdata_adjusted['FM_TravelModel']    = flatdata_adjusted['ForecastMethodology'].apply(lambda x : 1 if (x==3 or x==4) else 0)
flatdata_adjusted['FM_ProfJudgment']   = flatdata_adjusted['ForecastMethodology'].apply(lambda x : 1 if x==5 else 0)
flatdata_adjusted['FM_Unknown']        = flatdata_adjusted['ForecastMethodology'].apply(lambda x : 1 if x==6 else 0)


mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast \
                    + FC_PrincipalArterial \
                    + FC_MinorArterial \
                    + FC_CollectorLocal \
                    + FC_Unknown \
                    + AT_Rural \
                    + AT_Unknown \
                    + IT_NewRoad \
                    + IT_Unknown \
                    + DiffYear \
                    + FM_PopGrowthRates \
                    + FM_TravelModel \
                    + FM_ProfJudgment \
                    + FM_Unknown \
                    + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:              0.06346
Model:                       QuantReg   Bandwidth:                     0.06900
Method:                 Least Squares   Sparsity:                        1.721
Date:                Tue, 10 Jul 2018   No. Observations:                 3807
Time:                        17:17:52   Df Residuals:                     3792
                                        Df Model:                           14
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.1984      0.051     -3.917      0.000      -0.298      -0.099
AdjustedForecast      4.037e-07    3.7e-07      1.091      0.275   -3.22e-07    1.13e-06
FC_PrincipalArterial    -0.1000      0.029     -3.421      0.001      -0.157      -0.043
FC

### 13. Type of Forecaster

In [32]:
# Forecast Agency -- state DOT is reference

flatdata_adjusted['FA_DOT']        = flatdata_adjusted['ForecastingAgency'].apply(lambda x : 1 if x=='1-State DOT' else 0)
flatdata_adjusted['FA_MPO']        = flatdata_adjusted['ForecastingAgency'].apply(lambda x : 1 if x=='2-Metropolitan Planning Organization' else 0)
flatdata_adjusted['FA_Consultant'] = flatdata_adjusted['ForecastingAgency'].apply(lambda x : 1 if x=='5-Consultant' else 0)

mod = smf.quantreg('PercentError ~ \
                    AdjustedForecast \
                    + FC_PrincipalArterial \
                    + FC_MinorArterial \
                    + FC_CollectorLocal \
                    + FC_Unknown \
                    + AT_Rural \
                    + AT_Unknown \
                    + IT_NewRoad \
                    + IT_Unknown \
                    + DiffYear \
                    + FM_PopGrowthRates \
                    + FM_TravelModel \
                    + FM_ProfJudgment \
                    + FM_Unknown \
                    + FA_MPO \
                    + FA_Consultant \
                    + 1', data=flatdata_adjusted)

res05 = mod.fit(q=.05)
res50 = mod.fit(q=.5)
res95 = mod.fit(q=.95)

compare = pd.DataFrame({"05th" : res05.params, "50th" : res50.params, "95th" : res95.params})

print(res05.summary())
print(res50.summary())
print(res95.summary())

print("\n\nSummary of Parameters")
print(compare)

                         QuantReg Regression Results                          
Dep. Variable:           PercentError   Pseudo R-squared:              0.06570
Model:                       QuantReg   Bandwidth:                     0.06964
Method:                 Least Squares   Sparsity:                        1.784
Date:                Tue, 10 Jul 2018   No. Observations:                 3807
Time:                        17:26:04   Df Residuals:                     3790
                                        Df Model:                           16
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.1646      0.053     -3.095      0.002      -0.269      -0.060
AdjustedForecast      2.766e-07   3.91e-07      0.708      0.479   -4.89e-07    1.04e-06
FC_PrincipalArterial    -0.1001      0.032     -3.166      0.002      -0.162      -0.038
FC

### 14. Agency

In [24]:
# Jawad -- why are there only 6 projects from Ohio?  This doesn't seem right...

flatdata_adjusted['main_ProjectInformation.State'].value_counts()

OH    1581
MN    1320
WI     463
FL     425
MA      12
MI       6
Name: main_ProjectInformation.State, dtype: int64